In [ ]:
# tests with first data and repository



In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

project_dir = "/work/bd1062/b309257/oac-contrail-development/"
wd_ulrike = "/work/bd1033/b309022/liam/SMR_regional/"
multiplier = "1"

In [ ]:
# load repository dataset
ds_repo = xr.load_dataset(f"{project_dir}data/repository/emi_sa_reg.nc")
ds_repo

In [ ]:
ds = xr.load_dataset(f"{wd_ulrike}year_kero_mult{multiplier}_oa_input_variables_all_mean_2015_2018_yearly_cycle_liam.nc")

ds2 = xr.load_dataset("/work/bd1033/b309022/echam5_ccmod/run17702/run17702_201502.01_contr.nc")
ds3 = xr.load_dataset("/work/bd1033/b309022/echam5_ccmod/run17702/run17702_201502.01.nc")

In [ ]:
ds3.tradt

In [ ]:
ds2.isel(time=0).aps.plot()

In [ ]:
ds4 = xr.load_dataset("/work/bd1033/b309022/echam5_ccmod/run17701/run17701_201501.01.nc")

ds4

In [ ]:
# load CCMod dataset
ds = xr.load_dataset(f"{wd_ulrike}year_kero_mult{multiplier}_oa_input_variables_all_mean_2015_2018_yearly_cycle_liam.nc")
# ds = xr.load_dataset(f"{wd_ulrike}year_kero_regional_mult{multiplier}_oa_input_variables_all_mean_2021_2025_mean.nc")

# !!!TEMPORARY!!! change hybrid model level to pressure level
plev = (ds.hyam + ds.hybm * 101325) * 0.01  # [hPa] 
ds = ds.assign_coords(plev=("mlev", plev.data))
ds = ds.swap_dims({"mlev": "plev"})
ds = ds.drop_vars("mlev")
ds = ds.where(ds.plev >= 120., drop=True)  # remove data higher than 120 hPa (for now)

# change longitude to between -180 and +180
ds["lon"] = (ds.lon + 180) % 360 - 180
ds = ds.sortby("lon")

ds

In [ ]:
# FUNCTIONS

# lat-lon conversions for spherical Voronoi
def lonlat_to_xyz(lon, lat, radians=True):
    if not radians:
        lon = np.deg2rad(lon)
        lat = np.deg2rad(lat)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return np.stack((x, y, z), axis=-1)


def xyz_to_lonlat(pts, radians=True):
    x, y, z = pts[:, 0], pts[:, 1], pts[:, 2]
    lon = np.arctan2(y, x) % (2 * np.pi)
    lat = np.arcsin(z / np.sqrt(x**2 + y**2 + z**2))
    if not radians:
        lon = np.rad2deg(lon)
        lat = np.rad2deg(lat)
    return np.vstack((lon, lat)).T

In [ ]:
# calculate grid area using SphericalVoronoi
from scipy.spatial import SphericalVoronoi

# define points in lonlat and cartesian
lon_grid, lat_grid = np.meshgrid(ds.lon.values, ds.lat.values)
lon_flat = lon_grid.flatten()
lat_flat = lat_grid.flatten()
pts_ll = np.vstack((lon_flat, lat_flat)).T
pts_ct = lonlat_to_xyz(pts_ll[:, 0], pts_ll[:, 1], radians=False)

# calculate areas
sv = SphericalVoronoi(pts_ct)
sv.sort_vertices_of_regions()
r = 6371.0  # [km] assumed radius of the Earth
areas = sv.calculate_areas() * r ** 2  # [km2]

# add areas back into ds
areas_reshaped = areas.reshape(lon_grid.shape)
ds["areas"] = (("lat", "lon"), areas_reshaped)

ds

In [ ]:
# regrid ds_repo (flat) to CCMod grid using a histogram

# Extract data from ds_repo
distkm = ds_repo['distkm'].values
fuel = ds_repo['fuel'].values
lat_repo = ds_repo['lat'].values
lon_repo = ds_repo['lon'].values
plev_repo = ds_repo['plev'].values

# Extract coordinates from ds
lat_ds = ds['lat'].values
lon_ds = ds['lon'].values
plev_ds = ds['plev'].values

# Ensure coordinates are sorted
lat_ds_sorted = np.sort(lat_ds)
lon_ds_sorted = np.sort(lon_ds)
plev_ds_sorted = np.sort(plev_ds)

# Define the edges of the bins for lat, lon, and plev
lat_edges = np.concatenate(([lat_ds_sorted[0] - (lat_ds_sorted[1] - lat_ds_sorted[0]) / 2], 
                            (lat_ds_sorted[:-1] + lat_ds_sorted[1:]) / 2, 
                            [lat_ds_sorted[-1] + (lat_ds_sorted[-1] - lat_ds_sorted[-2]) / 2]))
lon_edges = np.concatenate(([lon_ds_sorted[0] - (lon_ds_sorted[1] - lon_ds_sorted[0]) / 2], 
                            (lon_ds_sorted[:-1] + lon_ds_sorted[1:]) / 2, 
                            [lon_ds_sorted[-1] + (lon_ds_sorted[-1] - lon_ds_sorted[-2]) / 2]))
plev_edges = np.concatenate(([plev_ds_sorted[0] - (plev_ds_sorted[1] - plev_ds_sorted[0]) / 2], 
                             (plev_ds_sorted[:-1] + plev_ds_sorted[1:]) / 2, 
                             [plev_ds_sorted[-1] + (plev_ds_sorted[-1] - plev_ds_sorted[-2]) / 2]))

# Use histogramdd to bin the data
hist_dist, edges = np.histogramdd((lat_repo, lon_repo, plev_repo), bins=(lat_edges, lon_edges, plev_edges), weights=distkm)
hist_fuel, _ = np.histogramdd((lat_repo, lon_repo, plev_repo), bins=(lat_edges, lon_edges, plev_edges), weights=fuel)

# create regridded ds
ds_rg = xr.Dataset()
ds_rg = ds_rg.assign_coords(ds.coords)
ds_rg["dist"] = xr.DataArray(hist_dist, coords=[lat_ds_sorted, lon_ds_sorted, plev_ds_sorted], dims=['lat', 'lon', 'plev'])
ds_rg["fuel"] = xr.DataArray(hist_fuel, coords=[lat_ds_sorted, lon_ds_sorted, plev_ds_sorted], dims=['lat', 'lon', 'plev'])

ds_rg

In [ ]:
# compare before and after re-gridding

import matplotlib.pyplot as plt

# Sum the "fuel" variable across all lat and lon for both datasets
ds_repo_gridded = xr.load_dataset(f"{project_dir}data/repository/emi_sa_reg_gridded.nc")
fuel_sum_repo = ds_repo_gridded['fuel'].sum(dim=['lat', 'lon'])
fuel_sum_repo_interp = ds_rg.fuel.sum(dim=['lat', 'lon'])

# Plot the vertical profiles
plt.figure(figsize=(10, 6))

plt.plot(fuel_sum_repo, fuel_sum_repo.plev, label='Original ds_repo', marker="s")
plt.plot(fuel_sum_repo_interp, fuel_sum_repo_interp.plev, label='Regridded ds_repo (histogram method)', linestyle='--', marker="o")

plt.gca().invert_yaxis()  # Invert y-axis to have plev decreasing upwards
plt.xlabel('Summed Fuel')
plt.ylabel('Pressure Level (hPa)')
plt.title('Vertical Profile of Fuel')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# add required data to ds_rg

ds_rg["areas"] = ds.areas
ds_rg["pPCF"] = ds.COVER_C_P_ACC
ds_rg["cccov"] = ds.cover_tot.isel(plev=0)  # independent of plev
ds_rg["RF"] = ds.rf.isel(plev=0)  # independent of plev

In [ ]:
# calculate CFDD

# for now. This is eventually aircraft design dependent
# check time as well
ds_rg["CFDD"] = (ds_rg.dist / len(ds_rg.time) / ds_rg.areas * ds_rg.pPCF).sum(dim="plev")
ds_rg

In [ ]:
# plot CFDD vs cccov

CFDD = ds_rg.CFDD.values.flatten()
cccov = ds_rg.cccov.values.flatten()

fig, ax = plt.subplots()
ax.scatter(CFDD, cccov * 100., marker="o", s=2)
# ax.set_ylim([0, 8])
ax.set_xlabel("CFDD")
ax.set_ylabel("cccov [%]")

In [ ]:
plt.hist(ds.COVER_CC_P_ACC.values.flatten())

In [ ]:
fig, ax = plt.subplots()
ax.scatter(ds_rg.cccov * 100., -ds_rg.RF, marker="o", s=2)
ax.set_xlabel("cccov [%]")
ax.set_ylabel("RF [W/m2]")

In [ ]:
plt.hist(ds_rg.RF.values.flatten())